### Étape 1 : Installation des bibliothèques requises

In [1]:
!pip install llama-index llama-index-llms-huggingface llama-index-llms-huggingface-api llama-index-embeddings-huggingface vllm transformers sentence-transformers pypdf


INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Étape 2 : Importation des classes nécessaires

In [2]:
from llama_index.core import VectorStoreIndex, Settings, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

### Étape 3 : Chargement des documents PDF

In [6]:
documents = SimpleDirectoryReader(input_files=["2307.14334.pdf","2307.15051.pdf"]).load_data()
print(f"Nombre de documents chargés : {len(documents)}")
print(documents[0].text[:1000])  # Aperçu

Nombre de documents chargés : 59
Towards Generalist Biomedical AI
Tao Tu∗, ‡, 1, Shekoofeh Azizi∗, ‡, 2,
Danny Driess2, Mike Schaekermann1, Mohamed Amin1, Pi-Chuan Chang1, Andrew Carroll1,
Chuck Lau1, Ryutaro Tanno2, Ira Ktena2, Basil Mustafa2, Aakanksha Chowdhery2, Yun Liu1,
Simon Kornblith2, David Fleet2, Philip Mansfield1, Sushant Prakash1, Renee Wong1, Sunny Virmani1,
Christopher Semturs1, S Sara Mahdavi2, Bradley Green1, Ewa Dominowska1, Blaise Aguera y Arcas1,
Joelle Barral2, Dale Webster1, Greg S. Corrado1, Yossi Matias1, Karan Singhal1, Pete Florence2,
Alan Karthikesalingam†, ‡,1 and Vivek Natarajan†, ‡,1
1Google Research,2Google DeepMind
Medicine is inherently multimodal, with rich data modalities spanning text, imaging, genomics, and more.
Generalist biomedical artificial intelligence (AI) systems that flexibly encode, integrate, and interpret
this data at scale can potentially enable impactful applications ranging from scientific discovery to care
delivery. To enable the dev

### Étape 4 : Initialisation du LLM HuggingFace (TinyLlama)

In [7]:
llm = HuggingFaceLLM(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    tokenizer_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    context_window=2048,
    max_new_tokens=256,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

### Étape 5 : Configuration du modèle d'embedding

In [8]:
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Étape 6 : Application des modèles à la configuration globale

In [9]:
Settings.llm = llm
Settings.embed_model = embed_model


### Étape 7 : Création de l’index des documents

In [10]:
index = VectorStoreIndex.from_documents(documents)


### Étape 8 : Persistance de l’index sur le disque

In [11]:
index.storage_context.persist(persist_dir="index_storage")


### Étape 9 : Interroger l’index

In [12]:
query_engine = index.as_query_engine()
question = "Quelles sont les techniques de prompting utilisées dans ces documents ?"
response = query_engine.query(question)
print(response)


This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


1. Prompting: The process of prompting is a technique used in natural language processing (NLP) to generate responses to user queries.
2. Dialogue Acts: Dialogue acts are a set of actions that a user can take in a dialogue.
3. Dialogue Modeling: Dialogue modeling is a technique used in NLP to generate responses to user queries.
4. Dialogue State Tracking: Dialogue state tracking is a technique used in NLP to keep track of the user's state in a dialogue.
5. Dialogue Act Recognition: Dialogue act recognition is a technique used in NLP to identify the dialogue act being used in a given text.
6. Dialogue Clustering: Dialogue clustering is a technique used in NLP to group similar dialogues together.
7. Dialogue Generation: Dialogue generation is a technique used in NLP to generate responses to user queries.
8. Dialogue Modeling: Dialogue modeling is a technique used in NLP to generate responses to user queries.
9. Dialogue State Tracking: Dialogue state tracking is a technique used in NLP t